In [1]:
'''
import zipfile
with zipfile.ZipFile("hw1__data.zip","r") as zip_ref:
    zip_ref.extractall("nwhw1_data")
'''    

In [250]:
import os
Task_name = 'nwhw1_data/hw1_data/Synthetic/5000'
data_root = "/home/p88101029/"
raw_data_base = os.path.join(data_root,Task_name)
os.makedirs(raw_data_base, exist_ok=True)

In [251]:
listoffolder=os.listdir(raw_data_base)

In [271]:
score_list = []
node_list = []
for element in listoffolder:
    if 'score' in element:
        score_list.append(element)
    if '_.' in element:
        node_list.append(element)
node_list=sorted(node_list)
score_list=sorted(score_list)

In [10]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [186]:
import networkx as nx
from torch_geometric.utils.convert import from_networkx
from sklearn.preprocessing import normalize
import torch.nn.functional as F

n = 5000
avg_degree = 50
p = 0.5

G = nx.powerlaw_cluster_graph(n, avg_degree, p)
bc=nx.betweenness_centrality(G)
data = from_networkx(G)
data.x = torch.eye(n,n)
data.y = F.normalize(torch.tensor([i for i in bc.values()]).reshape(-1,1), dim=0)

n = 5000
avg_degree = 50
p = 0.5

G2 = nx.powerlaw_cluster_graph(n, avg_degree, p)
bc2=nx.betweenness_centrality(G2)
data2 = from_networkx(G2)
data2.x = torch.eye(n,n)
data2.y = F.normalize(torch.tensor([i for i in bc2.values()]).reshape(-1,1), dim=0)


In [368]:
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import degree, add_self_loops
class GCNConv(MessagePassing):
    def __init__(self):
        super().__init__(aggr='add')  # "Add" aggregation (Step 5).


    def forward(self, x, edge_index):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]

        # Step 1: Add self-loops to the adjacency matrix.
        #edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))

        # Step 2: Linearly transform node feature matrix.


        # Step 3: Compute normalization.
        row, col = edge_index
        deg = degree(col, x.size(0), dtype=x.dtype)+1
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

        # Step 4-5: Start propagating messages.
        out = self.propagate(edge_index, x=x, norm=norm)

        return out

    def message(self, x_j, norm):
        # x_j has shape [E, out_channels]

        # Step 4: Normalize node features.
        return norm.view(-1, 1) * x_j


In [369]:
import random
def criterion(predicteddiff,targetdiff):
    loss = -(torch.sigmoid(targetdiff))*torch.log2(torch.sigmoid(predicteddiff))-(1-torch.sigmoid(targetdiff))*torch.log2(1-torch.sigmoid(predicteddiff))
    return torch.mean(loss)

def get_samples(n):
    list1=[]
    list2=[]
    for i in range(n*5):
        num1 = random.randint(0, n-1)
        num2 = random.randint(0, n-1)
        while num1 == num2:
            num2 = random.randint(0, n-1)
        list1.append(num1)    
        list2.append(num2)
    list1=torch.tensor(list1)
    list2=torch.tensor(list2)
    return list1, list2

In [375]:
import torch.nn as nn
from torch.nn import GRUCell
from torch.nn import Linear

class DRBC(nn.Module):
    def __init__(self, embedding_size, initialization_stddev, reg_hidden):
        super().__init__()
        self.embedding_size = embedding_size
        self.initialization_stddev = initialization_stddev
        self.reg_hidden = reg_hidden
        self.lin0 = Linear(1,self.embedding_size,bias=False)
        self.GCNConv = GCNConv()
        self.gru = GRUCell(self.embedding_size,self.embedding_size,bias=False)
        self.lin1 = Linear(self.embedding_size,self.reg_hidden,bias=False)
        self.lin2 = Linear(self.reg_hidden,1,bias=False)
        
    def forward(self, x, edge_index):
        deg = degree(edge_index[0]).cuda()
        deg = torch.reshape(deg,(-1,1))
        h0 = self.lin0(deg)
        h1 = F.relu(h0)
        h1 = F.normalize(h1, p=2, dim=1)
        hn2=self.GCNConv(h1,edge_index)
        h2 =self.gru(hn2,h1)
        h2 = F.normalize(h2, p=2, dim=1)
        hn3=self.GCNConv(h2,edge_index)
        h3 =self.gru(hn3,h2)
        h3 = F.normalize(h3, p=2, dim=1)
        hn4=self.GCNConv(h3,edge_index)
        h4 =self.gru(hn4,h3)
        h4 = F.normalize(h4, p=2, dim=1)
        hn5=self.GCNConv(h4,edge_index)
        h5 =self.gru(hn5,h4)
        h5 = F.normalize(h5, p=2, dim=1)
        hs=torch.stack((h1,h2,h3,h4,h5),dim=2)
        z=torch.max((hs),dim=2)
        declay1 = self.lin1(h5)
        declay2 = torch.relu(declay1)
        betw_pred = self.lin2(declay2)     
        return betw_pred

model = DRBC(embedding_size=64, initialization_stddev=1, reg_hidden=32)
model.to(device)

DRBC(
  (lin0): Linear(in_features=1, out_features=64, bias=False)
  (GCNConv): GCNConv()
  (gru): GRUCell(64, 64, bias=False)
  (lin1): Linear(in_features=64, out_features=32, bias=False)
  (lin2): Linear(in_features=32, out_features=1, bias=False)
)

In [376]:
optimizer = torch.optim.Adam(model.parameters(),lr=0.00001)  # Define optimizer.
torch.set_printoptions(precision=8)
def train(data):
    optimizer.zero_grad()  # Clear gradients.
    list1,list2 = get_samples(data.x.shape[0])
    out = model(data.x, data.edge_index)  # Perform a single forward pass.
    pred = out[list1]-out[list2]
    trag = data.y[list1]-data.y[list2]
    loss = criterion(pred, trag)  # Compute the loss solely based on the training nodes.
    loss.backward()  # Derive gradients.
    optimizer.step()  # Update parameters based on gradients.
    return loss

def valid(data):
    model.eval()
    list1,list2 = get_samples(data.x.shape[0])
    with torch.no_grad():
        out = model(data.x, data.edge_index)
        pred = out[list1]-out[list2]
        trag = data.y[list1]-data.y[list2]
        acc=torch.mean((torch.sign(pred)==torch.sign(trag)).float())
    return acc

count=0
bacc=0
for epoch in range(10000):
    data = data.to('cuda:0')
    loss = train(data)
    print("train", loss)
    data2 = data2.to('cuda:0')
    acc = valid(data2)
    print("valid", acc)
    if bacc < acc:
        bacc = acc
        count=0
    else:
        count+=1
    if count==50:
        break
    

train tensor(0.99998558, device='cuda:0', grad_fn=<MeanBackward0>)
valid tensor(0.96395999, device='cuda:0')
train tensor(0.99998504, device='cuda:0', grad_fn=<MeanBackward0>)
valid tensor(0.96307999, device='cuda:0')
train tensor(0.99998528, device='cuda:0', grad_fn=<MeanBackward0>)
valid tensor(0.96555996, device='cuda:0')
train tensor(0.99998498, device='cuda:0', grad_fn=<MeanBackward0>)
valid tensor(0.96399999, device='cuda:0')
train tensor(0.99998420, device='cuda:0', grad_fn=<MeanBackward0>)
valid tensor(0.96643996, device='cuda:0')
train tensor(0.99998373, device='cuda:0', grad_fn=<MeanBackward0>)
valid tensor(0.96503997, device='cuda:0')
train tensor(0.99998391, device='cuda:0', grad_fn=<MeanBackward0>)
valid tensor(0.96528000, device='cuda:0')
train tensor(0.99998379, device='cuda:0', grad_fn=<MeanBackward0>)
valid tensor(0.96528000, device='cuda:0')
train tensor(0.99998391, device='cuda:0', grad_fn=<MeanBackward0>)
valid tensor(0.96355999, device='cuda:0')
train tensor(0.9999

In [396]:
import numpy as np
import time
from scipy import stats

def topk_accuracy(k, pred, actu):
    pt=pred.topk(k, dim=0).indices.squeeze().cpu().numpy()
    at=actu.topk(k, dim=0).indices.squeeze().cpu().numpy()
    intersection = np.intersect1d(pt, at)
    count = len(intersection)
    return count
def return_rank(x):
    x=x.cpu().numpy()
    sorted_indices = np.argsort(x.squeeze())
    ranks = np.zeros_like(sorted_indices)
    ranks[sorted_indices] = np.arange(1, len(x) + 1)
    return(ranks)
def kendall_tau(x, y):
    x=return_rank(x)
    y=return_rank(y)
    n_samples = len(x)
    concordant_pairs = 0
    discordant_pairs = 0
    for i in range(n_samples):
        for j in range(i + 1, n_samples):
            if (x[i] < x[j] and y[i] > y[j]) or (x[i] > x[j] and y[i] < y[j]):
                discordant_pairs += 1
            else:
                concordant_pairs += 1
    tau = 2*(concordant_pairs - discordant_pairs) / (n_samples*(n_samples-1))
    return tau
def performance(model, edge_index, bc):
    n=bc.shape[0]
    onep=round(n*0.01)
    fivep=round(n*0.05)
    tenp=round(n*0.1)
    with torch.no_grad():
        start_time = time.time()
        out=model(1, edge_index)
        elapsed_time = time.time() - start_time
        oneacc=topk_accuracy(onep,out,bc)/onep*100
        fiveacc=topk_accuracy(fivep,out,bc)/fivep*100
        tenacc=topk_accuracy(tenp,out,bc)/tenp*100
        #kt=kendall_tau(out,bc)
        kt=stats.kendalltau(out.cpu(),bc.cpu())
        return(oneacc,fiveacc,tenacc,kt,elapsed_time)

In [387]:
from torch_geometric.utils import to_undirected
perfreco=[]
for (node_file,score_file) in zip(node_list,score_list):
    pairedge=[]
    with open('/home/p88101029/nwhw1_data/hw1_data/Synthetic/5000/'+node_file, 'r') as f:
        for line in f.readlines():
            pairedge.append(list(map(int,line.split())))
    score=[]        
    with open('/home/p88101029/nwhw1_data/hw1_data/Synthetic/5000/'+score_file, 'r') as f:
        for line in f.readlines():
            score.append([int(line.split()[0]),float(line.split()[1])])
    edge_index = torch.tensor(pairedge).t().contiguous()
    edge_index = to_undirected(edge_index)
    y = torch.tensor([y[1] for y in score], dtype=torch.float)
    edge_index=edge_index.to(device)
    y=y.to(device)
    perfreco.append(performance(model,edge_index,y))

In [391]:
print(perfreco)
print(np.mean(np.array(perfreco), axis=0))
print(np.std(np.array(perfreco), axis=0))

[(96.0, 82.39999999999999, 80.0, 0.47077031406281256, 0.15440034866333008), (90.0, 84.8, 79.80000000000001, 0.47369425885177036, 0.0035386085510253906), (92.0, 82.0, 77.4, 0.4683117423484697, 0.004873752593994141), (90.0, 80.4, 79.60000000000001, 0.4721133026605321, 0.005991458892822266), (88.0, 84.0, 78.0, 0.47655867173434685, 0.007806301116943359), (92.0, 81.2, 81.6, 0.47181644328865774, 0.0035245418548583984), (90.0, 83.6, 84.0, 0.49412170434086816, 0.003386259078979492), (92.0, 80.4, 82.39999999999999, 0.47381124224844967, 0.0039520263671875), (92.0, 81.6, 78.2, 0.46268549709941986, 0.0038955211639404297), (94.0, 86.8, 77.4, 0.47963128625725143, 0.0035767555236816406), (96.0, 85.2, 79.2, 0.480727825565113, 0.0037474632263183594), (90.0, 89.2, 80.4, 0.4809195439087818, 0.006331443786621094), (94.0, 84.0, 80.80000000000001, 0.4737741148229646, 0.0048675537109375), (94.0, 83.2, 79.60000000000001, 0.479750350070014, 0.004190206527709961), (92.0, 84.8, 80.4, 0.4691581516303261, 0.006150

In [300]:
for node_file in node_list:
    with open('/home/p88101029/nwhw1_data/hw1_data/Synthetic/5000/'+node_file, 'r') as f:
        pairedge=[]
        for line in f.readlines():
            pairedge.append(list(map(int,line.split())))
        num_rows = len(pairedge)
        header = '5000 '+str(num_rows)+'\n'
    with open('/home/p88101029/nwhw1_data/hw1_data/Synthetic/5000/'+node_file.replace("_.txt","_.net"), 'w') as f:
        f.writelines(header)
        for (x,y) in pairedge:
            f.write('{} {}\n'.format(x, y))

In [312]:
!tar czf archive.tar.gz /home/p88101029/nwhw1_data/hw1_data/Synthetic/5000/

tar: Removing leading `/' from member names


In [401]:
pairedge=[]
with open('/home/p88101029/nwhw1_data/hw1_data/youtube/com-youtube.txt', 'r') as f:
    for line in f.readlines():
        pairedge.append(list(map(int,line.split())))
score=[]        
with open('/home/p88101029/nwhw1_data/hw1_data/youtube/com-youtube_score.txt', 'r') as f:
    for line in f.readlines():
        score.append([int(line.split()[0].replace(":","")),float(line.split()[1])])
edge_index = torch.tensor(pairedge).t().contiguous()
edge_index = to_undirected(edge_index)
y = torch.tensor([y[1] for y in score], dtype=torch.float)
edge_index=edge_index.to(device)
y=y.to(device)
performance(model,edge_index,y)

(63.714864745792575,
 66.53566896940646,
 69.80588427072227,
 KendalltauResult(correlation=0.5370455517381877, pvalue=0.0),
 0.32272911071777344)